# Setup

In [1]:
import numpy as np
import os
import sklearn.metrics

import models
import util

SEED = 2021
np.random.seed(SEED)

# Stay in top-level directory for consistency
if '/src' in os.getcwd():
    os.chdir('..')

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [2]:
# Load data
xtrain, ytrain, xtest, ytest, xval, yval = util.load_preg_data(sim=True, onehots=True)
# Create binary labels
ytrain_early, ytrain_late, ytrain_preterm = util.preg_outcome_to_binaries(ytrain)
ytest_early, ytest_late, ytest_preterm = util.preg_outcome_to_binaries(ytest)
yval_early, yval_late, yval_preterm = util.preg_outcome_to_binaries(yval)

# Logistic Regression

## Train

In [3]:
lr_preterm = models.build_logreg()
models.fit_logreg(lr_preterm, xtrain, ytrain_preterm)
models.save_pickle(lr_preterm, 'models/lr_preterm')

/home/cove/Documents/S2021/cpsc464/fair-child/venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [4]:
# Example load and evaluate
del lr_preterm
lr_preterm = models.load_pickle('models/lr_preterm')
print(lr_preterm.score(xtest, ytest_preterm))
print(sklearn.metrics.classification_report(ytest_preterm, lr_preterm.predict(xtest), digits=4))

0.9695571955719557
              precision    recall  f1-score   support

           0     0.9646    0.9737    0.9691       532
           1     0.9744    0.9656    0.9700       552

    accuracy                         0.9696      1084
   macro avg     0.9695    0.9696    0.9696      1084
weighted avg     0.9696    0.9696    0.9696      1084



# Gradient Boosting

In [5]:
gb_preterm = models.build_gbdt()
models.fit_gbdt(gb_preterm, xtrain, ytrain_preterm, xval, yval_preterm)
models.save_pickle(gb_preterm, 'models/gb_preterm')

[100]	valid_0's auc: 0.995474	valid_0's binary_logloss: 0.610603
[200]	valid_0's auc: 0.995474	valid_0's binary_logloss: 0.542087
[300]	valid_0's auc: 0.995474	valid_0's binary_logloss: 0.484359
[400]	valid_0's auc: 0.995474	valid_0's binary_logloss: 0.435218
[500]	valid_0's auc: 0.995474	valid_0's binary_logloss: 0.39304
[600]	valid_0's auc: 0.995474	valid_0's binary_logloss: 0.356594
[700]	valid_0's auc: 0.995474	valid_0's binary_logloss: 0.324925
[800]	valid_0's auc: 0.995474	valid_0's binary_logloss: 0.297276
[900]	valid_0's auc: 0.995474	valid_0's binary_logloss: 0.273042
[1000]	valid_0's auc: 0.995474	valid_0's binary_logloss: 0.251728
[1100]	valid_0's auc: 0.995474	valid_0's binary_logloss: 0.232927
[1200]	valid_0's auc: 0.995474	valid_0's binary_logloss: 0.216301
[1300]	valid_0's auc: 0.995474	valid_0's binary_logloss: 0.201565
[1400]	valid_0's auc: 0.995474	valid_0's binary_logloss: 0.188479
[1500]	valid_0's auc: 0.998312	valid_0's binary_logloss: 0.175566
[1600]	valid_0's auc

# Neural nets

In [6]:
from importlib import reload
reload(models)

lrelu_preterm = models.build_NN_lrelu(input_len=xtrain.shape[1])  # Assuming xtrain is (batch_size, n_features)
lrelu_preterm.summary()

models.fit_NN_lrelu(lrelu_preterm, xtrain, ytrain_preterm, xval, yval_preterm)
models.save_NN(lrelu_preterm, 'models/lrelu_preterm')

Model: "NN_lrelu"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32)]              0         
_________________________________________________________________
lrelu1 (Dense)               (None, 70)                2310      
_________________________________________________________________
dropout (Dropout)            (None, 70)                0         
_________________________________________________________________
lrelu2 (Dense)               (None, 80)                5680      
_________________________________________________________________
dropout_1 (Dropout)          (None, 80)                0         
_________________________________________________________________
output (Dense)               (None, 1)                 81        
Total params: 8,071
Trainable params: 8,071
Non-trainable params: 0
________________________________________________________

INFO:tensorflow:Assets written to: models/lrelu_preterm/assets


INFO:tensorflow:Assets written to: models/lrelu_preterm/assets


In [8]:
del lrelu_preterm
lrelu_preterm = models.load_NN('models/lrelu_preterm')
lrelu_preterm.evaluate(xtest, ytest_preterm)  # Output is [loss, accuracy, auc]

34/34 [==============================] - 0s 2ms/step - loss: 102.3702 - accuracy: 0.5092


[102.37024688720703, 0.509225070476532]

In [9]:
selu_preterm = models.build_NN_selu(input_len=xtrain.shape[1])  # Assuming xtrain is (batch_size, n_features)
selu_preterm.summary()

models.fit_NN_selu(selu_preterm, xtrain, ytrain_preterm, xval, yval_preterm)
models.save_NN(selu_preterm, 'models/selu_preterm')

Model: "NN_selu"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32)]              0         
_________________________________________________________________
selu1 (Dense)                (None, 32)                1056      
_________________________________________________________________
alpha_dropout (AlphaDropout) (None, 32)                0         
_________________________________________________________________
selu2 (Dense)                (None, 32)                1056      
_________________________________________________________________
alpha_dropout_1 (AlphaDropou (None, 32)                0         
_________________________________________________________________
selu3 (Dense)                (None, 32)                1056      
_________________________________________________________________
alpha_dropout_2 (AlphaDropou (None, 32)                0   

INFO:tensorflow:Assets written to: models/selu_preterm/assets


In [10]:
del selu_preterm
selu_preterm = models.load_NN('models/selu_preterm')
selu_preterm.evaluate(xtest, ytest_preterm)  # Output is [loss, accuracy, auc]

34/34 [==============================] - 1s 2ms/step - loss: 0.1202 - accuracy: 0.1559 - auc: 0.9921


[0.12018220871686935, 0.15590405464172363, 0.9921338558197021]